# Clase Práctica: Embeddings, Búsqueda Semántica y LLMs Locales

**Duración estimada:** 60-75 minutos

En este notebook aprenderemos a:
1.  Entender la diferencia fundamental entre vectorización clásica (Bag of Words) y Embeddings.
2.  **Visualizar** embeddings en 2D para entender su agrupación semántica.
3.  Implementar un sistema de búsqueda semántica simple.
4.  Ejecutar un Gran Modelo de Lenguaje (LLM) localmente en Colab usando la GPU.
5.  Construir un **RAG (Retrieval Augmented Generation)** simple combinando todo lo anterior.

## 0. Configuración del Entorno (5 min)

Asegúrate de estar ejecutando este notebook con un entorno de ejecución T4 GPU en Google Colab.
`Entorno de ejecución > Cambiar tipo de entorno de ejecución > T4 GPU`

In [ ]:
# Verificamos disponibilidad de GPU
!nvidia-smi

In [ ]:

# Instalación de librerías necesarias
# matplotlib y scikit-learn son para visualización (PCA)
!pip install -q sentence-transformers transformers torch accelerate bitsandbytes langchain pypdf matplotlib scikit-learn

---

## 1. Embeddings vs Vectorización Clásica (15 min)

La vectorización clásica (como CountVectorizer o TF-IDF) se basa en la frecuencia de palabras exactas. Los Embeddings capturan el **significado semántico**.

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sentence_transformers import SentenceTransformer, util

# Frases de ejemplo con significado similar pero palabras distintas
frases = [
    "El perro corre en el jardín",
    "El canino juega en el patio",
    "The dog runs in the garden",  # Inglés
    "Me gusta programar en Python",
    "I love coding in Python",
    "La astronomía estudia las estrellas"
]

print("=== 1. Enfoque Clásico (Bag of Words) ===")
vectorizer = CountVectorizer()
vectors_bow = vectorizer.fit_transform(frases)

# Las columnas son palabras individuales. Notar la 'escasez' (muchos ceros).
df_bow = pd.DataFrame(vectors_bow.toarray(), columns=vectorizer.get_feature_names_out())
display(df_bow.head(3))

### El Poder de los Embeddings Multilingües

Ahora usaremos un modelo de Sentence Transformers entrenado para entender significado a través de múltiples idiomas.

In [ ]:
print("\n=== 2. Enfoque Semántico (Embeddings) ===")
# Usamos un modelo multilingüe: paraphrase-multilingual-MiniLM-L12-v2
model_multi = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

embeddings = model_multi.encode(frases)

# Calculamos similitud entre 'El perro...' y 'The dog...'
# Índices: 0 y 2
similitud = util.cos_sim(embeddings[0], embeddings[2])
print(f"Similitud 'Perro' vs 'Dog': {similitud.item():.4f}")

similitud_random = util.cos_sim(embeddings[0], embeddings[5])
print(f"Similitud 'Perro' vs 'Astronomía': {similitud_random.item():.4f}")

---

## 2. Visualización de Embeddings con PCA (10 min)

Los embeddings suelen tener cientos de dimensiones (ej. 384 o 768). Los humanos solo vemos en 3D. Usaremos **PCA (Principal Component Analysis)** para reducir esas dimensiones a 2 y poder graficarlo.

In [ ]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Generamos embeddings de una lista más variada
lista_conceptos = [
    "perro", "gato", "caballo", "vaca",   # Animales
    "dog", "cat", "horse", "cow",         # Animales EN
    "coche", "moto", "camión", "avión",   # Vehículos
    "car", "motorcycle", "truck", "plane", # Vehículos EN
    "manzana", "pera", "plátano", "fruta", # Comida
    "apple", "pear", "banana", "fruit"     # Comida EN
]

emb_conceptos = model_multi.encode(lista_conceptos)

# Reducir a 2 dimensiones
pca = PCA(n_components=2)
emb_2d = pca.fit_transform(emb_conceptos)

# Graficar
plt.figure(figsize=(10, 8))
plt.scatter(emb_2d[:, 0], emb_2d[:, 1])

for i, palabra in enumerate(lista_conceptos):
    plt.annotate(palabra, xy=(emb_2d[i, 0], emb_2d[i, 1]))

plt.title("Visualización de Embeddings con PCA")
plt.grid(True)
plt.show()

# NOTA: Observa cómo los conceptos similares (e idiomas) se agrupan juntos.

---

## 3. Manejo de Documentos y Búsqueda Semántica (15 min)

Chunking y recuperación de información.

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

texto_largo = """
La inteligencia artificial (IA) es un campo de la informática que busca crear sistemas capaces de imitar la inteligencia humana.
El aprendizaje profundo (Deep Learning) utiliza redes neuronales artificiales.
Los Grandes Modelos de Lenguaje (LLMs) como GPT-4, Claude o Llama 3 han revolucionado la interacción humano-máquina.
Hugging Face es una comunidad y plataforma de ciencia de datos que proporciona herramientas para trabajar con ML.
El procesamiento de lenguaje natural (NLP) permite a las máquinas entender texto y voz.
RAG (Retrieval-Augmented Generation) mejora las respuestas de los LLMs inyectando datos externos.
"""

# Chunking
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
chunks = text_splitter.split_text(texto_largo)

# Indexación (Crear embeddings solo una vez)
corpus_embeddings = model_multi.encode(chunks)

# Función de búsqueda
def buscar_informacion(pregunta, top_k=2):
    pregunta_emb = model_multi.encode(pregunta)
    hits = util.semantic_search(pregunta_emb, corpus_embeddings, top_k=top_k)
    
    resultados = []
    for hit in hits[0]:
        id_doc = hit['corpus_id']
        score = hit['score']
        resultados.append((chunks[id_doc], score))
    return resultados

# Prueba
print(buscar_informacion("¿Qué es RAG?")[0])
print(buscar_informacion("Plataforma de modelos de IA")[0])

---

## 4. LLMs Locales con Hugging Face (15 min)

Cargaremos un modelo optimizado (Quantized 4-bit) para generar texto.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)

print("Cargando LLM...")
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map="auto")
print("¡Modelo listo!")

---

## 5. RAG: Retrieval Augmented Generation (10 min)

Uniremos todo. Cuando el usuario hace una pregunta:
1.  Buscamos los chunks más relevantes.
2.  Construimos un prompt que incluya esa información.
3.  Le pedimos al LLM que responda USANDO esa información.

In [ ]:
def sistema_rag(pregunta):
    # 1. Recuperación (Retrieval)
    contexto_relevante = buscar_informacion(pregunta, top_k=1)
    texto_contexto = contexto_relevante[0][0] # Tomamos el texto del primer resultado
    
    # 2. Construcción del Prompt (Augmented)
    # Instruimos al modelo para que use el contexto
    prompt_final = f"""<|system|>
Usa la siguiente información para responder a la pregunta del usuario. Si no sabes, di que no sabes.
INFORMACIÓN: {texto_contexto}
<|user|>
PREGUNTA: {pregunta}
<|assistant|>
"""
    
    # 3. Generación (Generation)
    inputs = tokenizer(prompt_final, return_tensors="pt").to("cuda")
    outputs = model.generate(**inputs, max_new_tokens=150, temperature=0.7)
    respuesta = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Limpieza básica para extraer solo la parte del asistente
    if "<|assistant|>" in respuesta:
        return respuesta.split("<|assistant|>")[1].strip()
    return respuesta

# Probamos el sistema RAG
print("Pregunta: ¿Para qué sirve RAG?")
print("Respuesta LLM:", sistema_rag("¿Para qué sirve RAG?"))

print("\nPregunta: ¿Qué es Hugging Face?")
print("Respuesta LLM:", sistema_rag("¿Qué es Hugging Face?"))

---

## 6. Ejercicios Prácticos ("Your Turn") (15 min)

Ahora te toca a ti. Completa las siguientes celdas.

In [ ]:
# EJERCICIO 1: Nuevos Embeddings
# Crea una lista de palabras que incluyan "Tecnología", "Computadora", "Fútbol", "Deporte".
# Genera sus embeddings y calcula la similitud entre "Computadora" y "Fútbol".
# ¿Qué valor esperas obtener? (Alto/Bajo)

# Tu código aquí:


In [ ]:
# EJERCICIO 2: Prompting Creativo
# Modifica la función 'sistema_rag' o usa el modelo directamente para que responda 
# como si fuera un Pirata del siglo XVII.

prompt_pirata = "<|system|> Eres un pirata rudo. <|user|> Explícame qué es la IA. <|assistant|>"

# Tu código aquí para generar:
